In [1]:
# python version of https://migariane.github.io/TMLE.nb.html
import numpy as np
import pandas as pd
import statsmodels.api as sm

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def logit(p):
    return np.log(p) - np.log(1 - p)

def generate_data(n):
    np.random.seed(20)
    w1 = np.random.binomial(1, 0.5, n)        
    w2 = np.random.binomial(1, 0.65, n)    
    w3 = np.round(np.random.uniform(0, 4, n), 3)
    w4 = np.round(np.random.uniform(0, 5, n), 3)
    p_A = sigmoid(-0.4 + 0.2*w2 + 0.15*w3 + 0.2*w4 +0.15*w2*w4)
    
    A = np.random.binomial(1, p_A, n)
    
    p_y1 = sigmoid(-1 + 1 -0.1*w1 + 0.3*w2 + 0.25*w3+ 0.2*w4 + 0.15*w2*w4)
    p_y0 = sigmoid(-1 + 0 -0.1*w1 + 0.3*w2 + 0.25*w3 + 0.2*w4 + 0.15*w2*w4)
    Y1 = np.random.binomial(1, p_y1, n)
    Y0 = np.random.binomial(1, p_y0, n)
    
    Y = Y1 * A + Y0*(1-A)
    
    cols = ['w1', 'w2', 'w3', 'w4', 'A','Y', 'Y1', 'Y0']
    df = pd.DataFrame([w1, w2, w3, w4, A, Y, Y1, Y0]).T
    df.columns = cols
    return df
    

In [2]:
df = generate_data(30000)

true_psi = (df.Y1 - df.Y0).mean()
print('True Psi:', true_psi)
print(df.head())

True Psi: 0.19413333333333332
    w1   w2     w3     w4    A    Y   Y1   Y0
0  1.0  1.0  0.373  1.251  1.0  1.0  1.0  0.0
1  1.0  0.0  1.044  0.132  1.0  1.0  1.0  0.0
2  1.0  0.0  3.506  3.651  1.0  1.0  1.0  1.0
3  1.0  0.0  1.759  2.123  1.0  1.0  1.0  1.0
4  0.0  1.0  2.063  1.477  0.0  1.0  1.0  1.0


In [3]:
X_cols = ['w1', 'w2', 'w3', 'w4', 'A']
y_cols = ['Y']
X = sm.add_constant(df[X_cols], prepend=False)
mod = sm.OLS(df[y_cols],  X)
res = mod.fit()
res.summary()
biased_psi = res.params.A
print('Biased estimate of Psi:', biased_psi)

print('Amount of naive bias:', np.abs(biased_psi - true_psi))

naive_relative_bias = ((biased_psi - true_psi) / true_psi)*100
print('Relative naive bias:', naive_relative_bias, '%')

Biased estimate of Psi: 0.2072674786869555
Amount of naive bias: 0.013134145353622173
Relative naive bias: 6.765528169791642 %


In [4]:
# Intervene on A and add new columns to dataset
df['A0'] = 0
df['A1'] = 1

# Find Q0 by predicting the outcome Y using all covariates and treatment vars
X_cols = ['w1', 'w2', 'w3', 'w4', 'A']
y_cols = ['Y']
X = sm.add_constant(df[X_cols], prepend=False)
log_reg = sm.GLM(df[y_cols], X.values, family=sm.families.Binomial()).fit()
QAW = logit(log_reg.predict(X.values))
df['QAW'] = QAW

In [5]:
# Find QA1 and QA2
X_cols = ['w1', 'w2', 'w3', 'w4', 'A1']
X = sm.add_constant(df[X_cols], has_constant='add', prepend=False)
Q1W = logit(log_reg.predict(X.values))

X_cols = ['w1', 'w2', 'w3', 'w4', 'A0']
X = sm.add_constant(df[X_cols], has_constant='add', prepend=False)
Q0W = logit(log_reg.predict(X.values))

In [6]:
df['Q0W'] = Q0W
df['Q1W'] = Q1W

df['pY1'] = sigmoid(Q1W)
df['pY0'] = sigmoid(Q0W)

psi = (df['pY1'] - df['pY0'])
df['psi'] = psi
print('new psi estiamte:', psi.mean())
df.head()

new psi estiamte: 0.19925044639205422


,w1,w2,w3,w4,A,Y,Y1,Y0,A0,A1,QAW,Q0W,Q1W,pY1,pY0,psi
0,1.0,1.0,0.373,1.251,1.0,1.0,1.0,0.0,0,1,0.775654,-0.242047,0.775654,0.684743,0.439782,0.244961
1,1.0,0.0,1.044,0.132,1.0,1.0,1.0,0.0,0,1,-0.022999,-1.040700,-0.022999,0.494251,0.261015,0.233236
2,1.0,0.0,3.506,3.651,1.0,1.0,1.0,1.0,0,1,1.644920,0.627219,1.644920,0.838203,0.651859,0.186345
3,1.0,0.0,1.759,2.123,1.0,1.0,1.0,1.0,0,1,0.742651,-0.275051,0.742651,0.677575,0.431668,0.245908
4,0.0,1.0,2.063,1.477,0.0,1.0,1.0,1.0,0,1,0.364935,0.364935,1.382636,0.799414,0.590235,0.209180


In [7]:
# Find g by predicting the treatment A using covariates 
X_cols = ['w1', 'w2', 'w3', 'w4']
y_cols = ['A']
X = sm.add_constant(df[X_cols], has_constant='add', prepend=False)
log_reg = sm.GLM(df[y_cols], X.values, family=sm.families.Binomial()).fit()
g = logit(log_reg.predict(X.values))
df['g'] = g
df['pA1'] = sigmoid(g)

In [8]:
# Find clever covariates
df['Hgaw'] = df['A']/df['pA1'] - (1-df['A'])/(1-df['pA1'])
df['Hg1w'] = (1/df['pA1'])
df['Hg0w'] = -1/(1-df['pA1'])


In [9]:
# Find epsilon
y_cols = ['Y']
X_cols = ['Hgaw']
# no intercept, and use QAW as fixed offset
log_reg = sm.GLM(df[y_cols], df[X_cols], family=sm.families.Binomial(), offset=df['QAW']).fit()
epsilon = log_reg.params.Hgaw
df['epsilon'] = epsilon
df.head()

,w1,w2,w3,w4,A,Y,Y1,Y0,A0,A1,...,Q1W,pY1,pY0,psi,g,pA1,Hgaw,Hg1w,Hg0w,epsilon
0,1.0,1.0,0.373,1.251,1.0,1.0,1.0,0.0,0,1,...,0.775654,0.684743,0.439782,0.244961,0.318664,0.578999,1.727120,1.727120,-2.375289,0.000572
1,1.0,0.0,1.044,0.132,1.0,1.0,1.0,0.0,0,1,...,-0.022999,0.494251,0.261015,0.233236,-0.464349,0.385955,2.590978,2.590978,-1.628544,0.000572
2,1.0,0.0,3.506,3.651,1.0,1.0,1.0,1.0,0,1,...,1.644920,0.838203,0.651859,0.186345,0.978315,0.726774,1.375944,1.375944,-3.659969,0.000572
3,1.0,0.0,1.759,2.123,1.0,1.0,1.0,1.0,0,1,...,0.742651,0.677575,0.431668,0.245908,0.248717,0.561861,1.779800,1.779800,-2.282379,0.000572
4,0.0,1.0,2.063,1.477,0.0,1.0,1.0,1.0,0,1,...,1.382636,0.799414,0.590235,0.209180,0.646437,0.656207,-2.908727,1.523909,-2.908727,0.000572


In [10]:
# Update original estimator
Qstar1 = sigmoid(df['Q1W'] + df['epsilon']*df['Hg1w'])
Qstar0 = sigmoid(df['Q0W'] + df['epsilon']*df['Hg0w'])
df['Qstar1'] = Qstar1
df['Qstar2'] = Qstar0
df['debiased_psi'] = Qstar1 - Qstar0
TMLE_Psi = df['debiased_psi'].mean()
print('Debiased Psi:', TMLE_Psi)
print('TMLE bias:', np.abs(true_psi - TMLE_Psi))
naive_relative_TMLE_bias = ((TMLE_Psi - true_psi) / true_psi)*100
print('Relative TMLE bias:', naive_relative_TMLE_bias, '%')
print('This is a reduction in bias of :', naive_relative_bias-naive_relative_TMLE_bias, '%')

Debiased Psi: 0.19982321191644722
TMLE bias: 0.0056898785831139
Relative TMLE bias: 2.9309127316864183 %
This is a reduction in bias of : 3.8346154381052235 %
